In [ ]:
!pip3 install brevitas -q
# !pip install brevitas -q
# %xmode Verbose
# !pip install line_profiler -q
# %load_ext line_profiler

In [1]:
import torch
import brevitas
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import os
import cv2 as cv

# models_path = '/models_LN5'
# models_path = '/models_LN6'
# models_path = '/models_LN7'
# models_path = '/models_LN7_gciou'
models_path = '.'
# models_path = '/models_LN7_ciou'
# models_path = '/models_LN7_ciou_original'
# models_path = '/models_LN7_quant'

# dataset_local_path='/media/michal/HDD_Linux_2/DATASETS/Merged_dataset'
dataset_local_path='../DATASETS/Merged_dataset'
path_to_folds_state = 'folds_state_path_bbox.pkl'
local_g_drive = './'

# path directory with DAC SDC 2021 - current directory
drive_path = local_g_drive
# subdirectory with models
models_path = drive_path+models_path
# path to folds split data and it's state
path_to_folds_state = drive_path+path_to_folds_state

"""add path to allow import!!!"""
sys.path.append(drive_path)

# get device to execute 
use_cuda = torch.cuda.is_available()
# use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

import callbacks
import criterions
import preprocessing
import training
import utils
import metrics
import quantizers as quant
import networks
import schedulers

cuda


In [2]:

# image_shape = (100, 200, 3)
image_shape = (112, 208, 3)

after_load = preprocessing.numpy_to_torch_iou_params(device)
to_anchors_single = lambda *x: preprocessing.to_anchors_for_iou_loss(*x,False,False)
# to_anchors_multi = lambda *x: preprocessing.to_anchors_for_iou_loss(*x,True,True)
to_anchors_multi = lambda *x: preprocessing.to_anchors_for_iou_loss(*x,True,False)
# to_anchors_single = to_anchors_multi

anchors = [22,33,
            5,10,
            15,5
          ]
anchors = np.array(anchors, np.float32).reshape((-1,2))

# normalize anchors
anchors *= np.array([[image_shape[0]/340, image_shape[1]/640]])

qwi8 = quant.generalized_auto_fxp(bit_width=8, frac_part=8, signed=0.1, 
                                max_bit_width=8, min_bit_width=8, round_mode='floor',
                                trainable_signed=False, trainable_bit_width=False,
                                trainable_scale=False, dst='act')
qw8 = quant.generalized_auto_fxp(bit_width=8,frac_part=5, signed=0.9,
                                min_bit_width=4, max_bit_width=8, 
                                trainable_signed=False, trainable_bit_width=False,
                                dst='weight',)
qb8 = quant.generalized_auto_fxp(bit_width=8,frac_part=5, signed=0.9,
                                min_bit_width=4, max_bit_width=8, 
                                trainable_signed=False, trainable_bit_width=False,
                                dst='bias',)
qa = quant.generalized_auto_fxp(bit_width=8,frac_part=4, signed=0.9,
                                round_mode='floor',
                                min_bit_width=2, max_bit_width=8, 
                                trainable_signed=False, trainable_bit_width=False,
                                dst='act',)
quant_input = qwi8
quant_medium = (qw8,
                None,
                qa, # inter quant - before bn
                qa,# out quant - afeter 
                qb8)
quant_out = (qw8,None,qa,qa,qb8)

networks.SeparableConv2D.QUANT_BEFORE_BN = True
# networks.QuantBatchNorm.ALLOW_QUANTIZATION = False
networks.QuantBatchNorm.ALLOW_QUANTIZATION = True
net = networks.LittleNet7(anchors.shape[0], None, (None,None,None,None), (None,None,None,None), device=device)
# net = networks.LittleNet7(anchors.shape[0], quant_input, quant_medium, quant_out, device=device)

# pass example tensor
torch.cuda.empty_cache()
tensor = torch.rand((8,3,)+image_shape[:2]).to(device)
print("Input shape =",tensor.shape)
with torch.no_grad():
    result = net(tensor)

print("Result shape =",result.shape)

# get yolo paremeters
output_sizes = np.array(result.shape[2:][::-1])
p_num = networks.get_number_of_params(net)
del tensor
del result

print("Anchors: ")
print(anchors) 
print("Output sizes: ")
print(output_sizes) 
print("Number of parameters: ")
print(p_num) 

# CREATE GENERATORS
def numpy_to_tensor(X,y,device=device):
    return utils.data_to_tensor_v3(X,y,device)

grid_WH2 = image_shape[:2][::-1] // (2*output_sizes)
# create Transformer object
transform = preprocessing.Transformer(generator=np.random.default_rng(), 
                            noise=0.02, 
                            horizontal_flip=(), 
                            vertical_flip=(), 
                            rotate=(-15,15), 
                            # equalize_hist=0.05, 
                            blur=7,
                            scale=(0.8, 1.5),
                            translate=((-grid_WH2[0], grid_WH2[0]),
                                       (-grid_WH2[1], grid_WH2[1])),
                            HSV=0.005,
                            LAB=0.005,
                            YCrCb=0.005,
                            )
# create generators without data => generators templates
train_generator = preprocessing.YoloDataGenerator(
                            dataset_local_path,
                            input_shape=image_shape,
                            anchors=anchors,
                            images_labes=[],# [(img_path, bbox_path),] 
                            batch_size=64,
                            name='TrainGenerator', 
                            augmentator=transform,
                            output_size=output_sizes,
                            after_load=after_load,
                            # depends on model
                            bbox_to_anchors=to_anchors_multi,
                            )
val_generator = preprocessing.YoloDataGenerator(
                            dataset_local_path,
                            input_shape=image_shape,
                            anchors=anchors,
                            images_labes=[], 
                            batch_size=64,
                            name='ValGenerator', 
                            augmentator=None,
                            output_size=output_sizes,
                            after_load=after_load,
                            bbox_to_anchors=to_anchors_single,
                            # bbox_to_anchors=to_anchors_multi,
                            )
test_generator = preprocessing.YoloDataGenerator(
                            dataset_local_path,
                            input_shape=image_shape,
                            anchors=anchors,
                            images_labes=[], 
                            batch_size=64,
                            name='TestGenerator', 
                            augmentator=None,
                            output_size=output_sizes,
                            after_load=after_load,
                            bbox_to_anchors=to_anchors_single,
                            # bbox_to_anchors=to_anchors_multi,
                            )

# PATHS FORMATS
# paths format is (fold_idx, '{}') - to be possible next format
checkpoint_format_path = models_path+'/model_chp_fold_{}_epoch_{}.pt'
best_format_path = models_path+'/model_best_fold_{}.pt'
best_iou_format_path = models_path+'/model_best_iou_fold_{}.pt'
last_format_path = models_path+'/model_last_fold_{}.pt'
global_last_path = models_path+'/network_model_last.pt' 


Input shape = torch.Size([8, 3, 112, 208])
Result shape = torch.Size([8, 15, 7, 13])
Anchors: 
[[ 7.247059  10.725    ]
 [ 1.6470588  3.25     ]
 [ 4.9411764  1.625    ]]
Output sizes: 
[13  7]
Number of parameters: 
243919


In [3]:
# create_trainer_again = True
create_trainer_again = False

lr = 0.1
momentum = 0.9
# optimizer = torch.optim.Adadelta(net.parameters(), lr=lr, weight_decay=0.0001)
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=momentum)
# optimizer = torch.optim.Adam(net.parameters(), lr=lr)

if create_trainer_again:
    # init all obejcts once

    # criterion
    loss_criterion = criterions.YoloIOULoss(anchors, 
                                            input_shape=image_shape, 
                                            # weights=[20,1],
                                            weights=[1,1],
                                            iou_mode='gciou')
                                            # iou_mode='ciou')
    criterion = criterions.Regularization(loss_fcn=loss_criterion,
                                          regularization_loss=criterions.mean_sqare_value,
                                          weights=[1.0, 0.00001])

    # create metrics
    iou = metrics.SingleObjectIOUsBasedMetrics(anchors.copy(),image_shape, device=device)
    giou, diou, ciou = iou.get_gdciou()

    reg_loss_metric = metrics.ProxyAttributeMetric(criterion, 'reg_loss')
    loss_validity_metric = metrics.ProxyAttributeMetric(criterion.loss_fcn, 'loss_validity')
    loss_iou_metric = metrics.ProxyAttributeMetric(criterion.loss_fcn, 'loss_iou')

    # create callbacks
    checkpoint = callbacks.Checkpoint(checkpoint_format_path.format(-1,'{}'))
    best = callbacks.SaveBest(best_format_path.format(-1),monitored=['loss','val_loss'], multipliers=[-1, -1])
    best_iou = callbacks.SaveBest(best_iou_format_path.format(-1),monitored=['val_iou'], multipliers=[1])
    last = callbacks.SaveLast(last_format_path.format(-1))
    last_global = callbacks.SaveLast(global_last_path)
    early_stopping = callbacks.EarlyStopping(filter_size=5, threshold=0.3)
    # plot_clb = callbacks.PlotHistory(5)

    # create scheduler
    scheduler_ = schedulers.LossDependentScheduler(mul=1.3, div=2.0, init_loss=-np.inf, init_lr=lr, lr_min=1e-5, lr_max=1.0)
    # scheduler_ = schedulers.BaseScheduler()
    # group metrics and callbacks
    metrics_dict = {'r_l':reg_loss_metric,'obj_l':loss_validity_metric,'iou_l':loss_iou_metric,'iou': iou, 'giou':giou, 'diou':diou, 'ciou':ciou}
    callbacs_list = [checkpoint,
                     best,best_iou, 
                     last,last_global,
                     early_stopping,
                    #  plot_clb, 
                     ]

    trainer = training.Trainer(net,
                            criterion=criterion,
                            optimizer=optimizer,
                            name='Trainer',
                            metrics=metrics_dict,
                            callbacks=callbacs_list,
                            checkpoint_clb=checkpoint,
                            best_clb=best,
                            best_iou_clb=best_iou,
                            last_clb=last,
                            last_global_clb=last_global,
                            early_stopping=early_stopping,
                            # plotter=plot_clb,
                            info=['LittleNet7 quant, gciou, with input '+str(image_shape)],
                            anchors_wh=anchors,
                            scheduler=scheduler_,
                            fold_state=-1)

    torch.save(trainer.get_state(), global_last_path)
    
    print("Trainer object created and saved at", global_last_path)


In [5]:
# SPLIT DATASET ONTO TESTSET and TRAIN-VAL FOLDS
# DO IT ONCE
split_again = False

if split_again:
    # get and split data
    dataset = utils.get_all_imgs_gts(dataset_local_path)
    training_dataset, test_dataset = utils.train_test_split(dataset, 0.4)

    # create folds split for cross validation
    folds_of_dicts = utils.split_into_folds(training_dataset, 5)
    folds = utils.folds_dict_to_folds_list(folds_of_dicts)
    # train set for final evaluation
    test_set = utils.dataset_to_list(test_dataset)
        
    # create folds wrapper 
    folds_state = training.FoldsState(folds=folds, state=0, test_set=test_set)

    training.save_folds(folds_state, path_to_folds_state)

    print("Data splited into folds")

In [ ]:
# d = {'folds':{},
#      'test':[]}
# for idx, f in enumerate(folds_state.folds):
#     Ld = {i:{'path':p,'bbox':dict(zip(['l','t','r','b'],np.round(utils.xcycwh_to_ltrb(bbox.copy().reshape(1,4))).flatten().astype(int).tolist()))} for i, (p,bbox) in enumerate(f)}
#     d['folds'][idx] = Ld
#     pass
    
# d['test'] = {i:{'path':p,'bbox':dict(zip(['l','t','r','b'],np.round(utils.xcycwh_to_ltrb(bbox.copy().reshape(1,4))).flatten().astype(int).tolist()))} for i, (p,bbox) in enumerate(folds_state.test_set)}

# for k,v in d['folds'].items():
#     print(k, len(v))
#     print(v[0])
# print(d['test'][0])

# import json

# with open('splited_data_set.json', 'w') as f:
#     as_str = json.dumps(d,indent=True)
#     f.write(as_str)

    

In [4]:
# load state 
folds_state = training.load_folds(path_to_folds_state)
# state_dict = torch.load(global_last_path, map_location=torch.device(device))
state_dict = torch.load('../trained_models/float_gciou.pt', map_location=torch.device(device))
# state_dict = torch.load(best_format_path.format(0), map_location=torch.device(device))
# state_dict = torch.load('../network_model_last.pt', map_location=torch.device(device))
# state_dict = torch.load(checkpoint_format_path.format(0, 69), map_location=torch.device(device))
trainer = training.load_trainer(net, optimizer, state_dict, device)
# we = state_dict['model']
# trainer.additional_state['scheduler'] = schedulers.LossDependentScheduler(mul=1.3, div=2.0, init_loss=-np.inf, init_lr=lr, lr_min=1e-4, lr_max=1.0)
# trainer.criterion.loss_fcn.weights = [1.0,2.0]
# trainer.criterion.loss_fcn.input_shape = torch.tensor(image_shape)
# trainer.metrics['iou'].img_shape = image_shape
# trainer.criterion.regularization_loss = criterions.NearestQuantLoss(8, 2, signed=True, narrow_range=False)
# trainer.criterion.regularization_loss.norm = torch.abs
# trainer.criterion.regularization_loss.reduction = torch.max

# trainer.criterion.loss_fcn.iou_mode = 'gciou'
# # additional notes
# trainer.additional_state['info'].append(str(trainer.epoch)+' - scheduler LD')
# trainer.additional_state['info'] = trainer.additional_state['info'][:-1]
# trainer.additional_state['info'].append(str(trainer.epoch)+' - ciou alpha always 1 validity loss from 1 to 100')
# torch.save(trainer.get_state(), global_last_path)
# trainer.additional_state['best_iou_quant_clb'].best *= 0

# trainer.additional_state['info'].append(str(trainer.epoch)+' - start trainable fixedpoint quantization')
print(trainer.additional_state.keys())
# print(trainer.additional_state['info'])

<All keys matched successfully>
dict_keys(['checkpoint_clb', 'best_clb', 'best_iou_clb', 'last_clb', 'last_global_clb', 'early_stopping', 'info', 'anchors_wh', 'scheduler', 'fold_state'])


In [ ]:
# test_generator.images_labes = folds_state.test_set
# test_generator.batch_size = 1

# for i in range(min(200,len(test_generator))):
#     img, y, bbox = test_generator.__getitem__(i,True)
#     img = img[0,...].cpu().numpy()
#     img = img.reshape((3,-1)).T.reshape((112,208,3)).copy()*255
#     img = img.astype(np.uint8)
#     bbox = bbox[0,:]
#     print(img.shape)
#     img = utils.draw_bbox(img,bbox)
#     plt.imshow(img)
#     plt.show()


In [ ]:
# h = {k+'_ciou':v[:43] for k,v in trainer.history.items()}
# h.update(trainer.history)
# plt.grid(True)
# hist = h
# keys = [k for k in hist.keys() if 'val_' not in k and '_ciou' not in k ]

# for k in keys:
#     v1 = hist[k]
#     v2 = hist['val_'+k]
#     v3 = hist[k+'_ciou']
#     v4 = hist['val_'+k+'_ciou']
    
#     v1 = [v.item() if isinstance(v, torch.Tensor) else v for v in v1]
#     v2 = [v.item() if isinstance(v, torch.Tensor) else v for v in v2]
#     v3 = [v.item() if isinstance(v, torch.Tensor) else v for v in v3]
#     v4 = [v.item() if isinstance(v, torch.Tensor) else v for v in v4]

#     size = min(len(v1),len(v2),len(v3),len(v4))
#     x = np.arange(0,size)+1

#     plt.plot(x, v1[:size], label=k)
#     plt.plot(x, v2[:size], label='val_'+k)
#     plt.plot(x, v3[:size], label=k+'_ciou')
#     plt.plot(x, v4[:size], label='val_'+k+'_ciou')
#     plt.legend()
#     plt.xlabel("Epoch")
#     plt.ylabel(k+' value')
#     plt.title("History of '{}'".format(k))
#     plt.grid(True)
#     plt.show()

# utils.plot_history(h)

In [ ]:
# HERE IS MAIN CROSS-VALIDATION LOOP

epochs_limit = 350
if device == torch.device('cpu') and False:
    train_generator.batch_size = 2
    val_generator.batch_size = 4
    test_generator.batch_size = 4
else:
    train_generator.batch_size = 8
    # train_generator.batch_size = 8
    val_generator.batch_size = 8
    test_generator.batch_size = 8

update_period = 1 + 48*10 // train_generator.batch_size
# update_period = 4

# get fold state
current_fold = folds_state.state

for fold_idx in range(current_fold, len(folds_state)):
    # if new fold
    if fold_idx != trainer.additional_state['fold_state']:
        print("FOLD", fold_idx, "begin ")
        # set fold state
        trainer.additional_state['fold_state'] = fold_idx
        trainer.reset()
        # save reseted model
        torch.save(trainer.get_state(), global_last_path)
    else:
        print("FOLD", fold_idx, "continue")

    # get fold data split
    train_data, val_data = folds_state.__getitem__(fold_idx, train_folds=4)
    # set callbacks path apropriate to current fold
    trainer.additional_state['checkpoint_clb'].formatable_path = checkpoint_format_path.format(fold_idx, '{}')
    trainer.additional_state['best_clb'].path = best_format_path.format(fold_idx)
    trainer.additional_state['best_iou_clb'].path = best_iou_format_path.format(fold_idx)
    trainer.additional_state['last_clb'].path = last_format_path.format(fold_idx)
    trainer.additional_state['last_global_clb'].path = global_last_path
    
    train_generator.images_labes = train_data
    val_generator.images_labes = val_data
    test_generator.images_labes = folds_state.test_set

    # train model up to epochs limit
    hist = trainer.continue_fit(train_generator=train_generator, 
                                epochs=epochs_limit, 
                                val_generator=val_generator,
                                update_period=update_period)
    
    # evaluate
    loss, metrics = trainer.score(test_generator)
    print("Test evaluation:",loss, metrics)
    # update fold state and save
    folds_state.state += 1
    training.save_folds(folds_state, path_to_folds_state)
    # save model
    torch.save(trainer.get_state(), global_last_path)
    print("FOLD", fold_idx, "finished")

In [ ]:
# compute gciou loss metric history
# H = trainer.history.copy()
# H = state_dict['history']
H = src_dict['history']
gciou = np.array(H['ciou']) + np.array(H['giou']) - np.array(H['iou'])
H['gciou'] = gciou.tolist()
val_gciou = np.array(H['val_ciou']) + np.array(H['val_giou']) - np.array(H['val_iou'])
H['val_gciou'] = val_gciou.tolist()

# utils.plot_history(H, models_path + "/8_bit_quant_hist_of_{}.png")
utils.plot_history(H, models_path + "/LN_smaller_hist_of_{}.png")
utils.plot_history(H)
# print(np.max(H['val_iou'][-5:]))
# print(np.max(H['iou'][-5:]))
# print(src_dict['additional_state']['info'])


In [7]:
test_generator.batch_size = 128+64
val_generator.batch_size = 128+64

# test_generator.images_labes = folds_state.test_set
# losses_test, metrics_test = trainer.score(test_generator)
# losses_val, metrics_val = trainer.score(val_generator)


test_generator.images_labes = folds_state.test_set[:3000]
losses_test_3000, metrics_test_3000 = trainer.score(test_generator)

# print("For val:",losses_val, metrics_val)
# print("For test:",losses_test, metrics_test)
print("For test_3000:",losses_test_3000, metrics_test_3000)

Score [████████████████████] [16/16] loss=0.25813 r_l=0.38533 obj_l=0.01868 iou_l=0.23945 iou=0.66819 giou=0.59031 diou=0.61082 ciou=0.61081 time=30[s]:92[ms]]
For test_3000: 0.258128598690033 {'r_l': 0.38532575964927673, 'obj_l': 0.018677696585655212, 'iou_l': 0.23944710195064545, 'iou': 0.6681918849945068, 'giou': 0.5903083944320677, 'diou': 0.6108209228515623, 'ciou': 0.6108059921264648}


In [8]:
test_generator.images_labes = folds_state.test_set[:3000]
test_generator.batch_size = 128+64
losses_test_3000, metrics_test_3000 = trainer.score(test_generator)
print("For val:",losses_val, metrics_val)

Score [████████████████████] [16/16] loss=0.26379 r_l=0.91175 obj_l=0.01679 iou_l=0.24699 iou=0.66180 giou=0.58315 diou=0.60443 ciou=0.60442 time=31[s]:646[ms]
For val: 0.26734587816304334 {'r_l': 0.9117961525917053, 'obj_l': 0.01681588776409626, 'iou_l': 0.250522255897522, 'iou': 0.6528222642133054, 'giou': 0.5669894416373907, 'diou': 0.5894029177551819, 'ciou': 0.5893876987160576}


In [ ]:
visual_test = True

if visual_test:
    net = trainer.model
    # gen = val_generator
    gen = test_generator
    # gen.images_labes = folds_state.test_set
    gen.images_labes = dst_name_bbox[:16]
    # print(dst_name_bbox[:16])
    gen.batch_size = 16
    scale = [[640/200,360/100,640/200,360/100]]
    # images_labels = [(name_img, name_xml),(name_img, name_xml),(name_img, name_xml),(name_img, name_xml),]
    # gen.images_labes = utils.convert_2x_path_into_path_bbox(dataset_local_path,images_labels)
    # gen.batch_size = 4

    index = np.random.randint(0, len(gen))
    imgs, y_refs, bboxs = gen.__getitem__(index, True)

    # imgs 
    np.set_printoptions(edgeitems=100, linewidth=200)
    
    # net inference
    with torch.no_grad():
        y_pred = net2(imgs)[0]
        # y_pred = net(imgs)
        # outs = net2.multioutput_forward(imgs)
        # y_pred = outs[-1][0]
        # print(y_pred[0,...].detach().numpy())
    
    # first = y_pred[0,...].reshape(5,3*13*7)
    # pos = torch.argmax(first[:1,:]).detach().numpy()
    # vals = first[1:,pos].clone().detach().numpy()
    # # sigm
    # v = vals[:2]
    # e = np.exp(v)
    # s = e / (e+1)
    # xy = np.array([5,1]) + s
    # xy_ = xy*np.array([200/13,100/7], dtype=np.float32)

    # wh = np.exp(vals[-2:])
    # wh_ = wh*anchors[0,:]
    # lt = xy_-wh_/2
    # rb = lt+wh_
    # lt_ = lt*np.array(scale)[0,:2]
    # rb_ = rb*np.array(scale)[0,:2]
    # print("Manually computed:")
    # print(vals)
    # print(xy, xy_, wh, wh_)
    # print(lt,rb)
    # print(lt_,rb_)
    # print(lt_.astype(np.int32),rb_.astype(np.int32))



    y_pred[:,:3*anchors.shape[0],:,:] = torch.sigmoid(y_pred[:,:3*anchors.shape[0],:,:])

    # get bbox of prediction
    bbox_pred = utils.yolo_outputs_to_single_bbox_v3(y_pred, torch.tensor(anchors, device=device), image_shape)
    bbox_pred = np.round(bbox_pred[:,1:].cpu().detach().numpy().reshape(-1,4)).astype(np.int)
    bbox_pred_all = bbox_pred
   
    # get bbox of y_refs
    bbox_ref = utils.ltrb_to_xcycwh(y_refs[0].clone())
    # print(bbox_ref)
    bbox_ref = np.round(bbox_ref.cpu().detach().numpy().reshape(-1,4)).astype(np.int)
    bbox_ref_all = bbox_ref
    # draw
    for idx in range(0,bboxs.shape[0]):
        img = imgs[idx,:,:,:].clone().permute(1,2,0).cpu().detach().numpy()*255
        img = img.astype(np.uint8)[:,:,::-1]
        # original
        img_original_marked = utils.draw_bbox(img.copy(), bboxs[idx,:], color=(0,255,0))
        # plt.imshow(img_original_marked)
        # plt.show()
        bbox_pred = bbox_pred_all[idx,:]
        # mark prediction
        img_original_marked = utils.draw_bbox(img_original_marked, bbox_pred, color=(0,0,255))
        # plt.imshow(img_original_marked)
        # cv.imwrite('SO_img.png', img_original_marked[:,:,::-1])
        # plt.show()
        bbox_ref = bbox_ref_all[idx,:]
        # mark reference bbox 
        # img_original_marked = utils.draw_bbox(img_original_marked, bbox_ref, color=(100,0,0))
        plt.imshow(img_original_marked)
        plt.show()


    # cv.imwrite(name_out, img_original_marked)
    plt.gray()
    
    print(y_refs[0])
    ref = (y_refs[0]*torch.tensor(scale)).round()
    print(ref)
    iou = metrics.iou_based_metric(
                                utils.xcycwh_to_ltrb(torch.tensor(np.round(bbox_pred_all.astype(np.float32)*np.array(scale))).floor()),
                                ref,
                                mode='gdciou')
    
    for i, (n,m) in enumerate(zip(['iou','giou','diou','ciou'],list(iou))):
        print(n,":",torch.mean(m).item())

        
    # IMG = utils.batch_masks_to_img(y_pred, image_shape, anchors)[0]
    # plt.imshow(IMG)
    # plt.show()
    # IMG = utils.batch_masks_to_img(y_refs, image_shape, anchors)[0]
    # plt.imshow(IMG)
    # plt.show()
    # print(y_pred)
    # print(y_refs)



In [14]:
import json
# CREATE TEST EVAL SET
subset = folds_state.test_set[:3000]
d = {}
generator = preprocessing.YoloDataGenerator(
                            dataset_local_path,
                            input_shape=image_shape,
                            anchors=anchors,
                            images_labes=subset, 
                            batch_size=1,
                            name='Generator', 
                            augmentator=None,
                            output_size=output_sizes,
                            after_load=after_load,
                            # bbox_to_anchors=to_anchors_single,
                            bbox_to_anchors=to_anchors_single,
                            )

dst_dir_path = '../EVAL/eval_images/'
# for i,(path,bbox) in enumerate(subset):
for i in range(len(generator)):
    img,Y = generator[i]
    img = img.reshape(3,-1).numpy().T.reshape((112,208,3))
    img = (img*255).astype(np.uint8)
    bbox = np.round(Y[0].reshape(-1).numpy()).astype(int).flatten().tolist()
    
    new_path = 'images/img_'+(str(i).zfill(4))+'.png'
    dst = os.path.join(dst_dir_path,new_path)
#     bbox = np.round(utils.xcycwh_to_ltrb(bbox.copy().reshape((1,4)))).astype(int).flatten().tolist()
    bbox = {k:v for k,v in zip('ltrb',bbox)}
    d[str(i)] = {'path':new_path,'bbox':bbox}
    print('\r',dst, bbox,end='')
    cv.imwrite(dst, img)
    
# print(d)
with open(os.path.join(dst_dir_path,'gt.json'),'w') as f:
    f.write(json.dumps(d, indent=4,))

 eval_images/images/img_2999.png {'l': 135, 't': 20, 'r': 199, 'b': 57}}}al_images/images/img_0310.png {'l': 65, 't': 4, 'r': 76, 'b': 36}

In [6]:
import quantizers, json
def get_dataset(path):
    with open(os.path.join(path,'gt.json')) as f:
        ds = json.loads(f.read())
    
    paths = []
    ltrb = []
    for v in ds.values():
        paths.append(os.path.join(path,v['path']))
        bbox = v['bbox']
        ltrb.append([bbox['l'],bbox['t'],bbox['r'],bbox['b']])

    return paths, ltrb

paths, gts = get_dataset('../EVAL/eval_images')
bb = np.array(gts).astype(np.float32)
bb = utils.ltrb_to_xcycwh(bb)
ds = [(p,b) for p,b in zip(paths,bb)]

generator = preprocessing.YoloDataGenerator(
                            # dataset_local_path,
                            '',
                            input_shape=image_shape,
                            anchors=anchors,
                            # images_labes=folds_state.test_set[:1], 
                            images_labes=ds, 
                            batch_size=1,
                            name='Generator', 
                            augmentator=None,
                            output_size=output_sizes,
                            after_load=after_load,
                            bbox_to_anchors=to_anchors_single,
                            )
# folds = training.load_folds(path_to_folds_state)
# net.load_state_dict(sd,strict=True)
# sd = torch.load('../weights_quant_gciou.pt',map_location=torch.device(device))
# net.load_state_dict(sd)
networks.SeparableConv2D.QUANT_BEFORE_BN = True

iou = metrics.SingleObjectIOUsBasedMetrics(anchors.copy(),image_shape, device=device)

generator.batch_size = 1
# generator.images_labes = folds_state.test_set[:1]
iou_results = 0.0
torch.set_printoptions(linewidth=3000,edgeitems=15*13*7)
np.set_printoptions(linewidth=3000,edgeitems=15*13*7)

with torch.no_grad():
    for i in  range(len(generator)):
        print('\r',i,'/',len(generator),end='')
        X,Y = generator.__getitem__(i)
        y_pred = net(X)
        iou_results += iou(y_pred,Y)

print('\r',len(generator),'/',len(generator))
iou_results /= len(generator)

print(iou_results)


 3000 / 3000000 30003000/ 30003000 3000
0.5697525913912298


In [ ]:
test_generator.images_labes = folds_state.test_set[:300]
test_generator.batch_size = 1
loss, metrics_ = trainer.score(test_generator)
print("Test evaluation:",loss, metrics_)


In [3]:
folds_state = training.load_folds(path_to_folds_state)
print(len(folds_state.test_set))
train, val = folds_state.__getitem__(0, train_folds=4)
print(len(train))
print(len(val))

97969
117409
29417
